In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# print(pd.__version__)
import leaguedata as ld

#Progress bars wooo
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Markdown

In [2]:
matches = ld.get_all_matches()
print(len(matches))

7620


In [3]:
print(matches[1].info()['Address'].values)

['http://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH1/30054?gameHash=055b17da8456fdc8']


In [4]:
num_games = 100
data = None
# prev_structures = test.prev_kills(time)
# prev_structures_5_mins = prev_structures.loc[prev_structures['Time'] > (time-5)]
# print(prev_structures)
# print(prev_structures_5_mins)
# print(len(prev_structures_5_mins.index)/5)
# print(test.kill_freq(time))
# print(test.prev_kills(20)[2])
# print(test.game_state(5))

for num in range(num_games):
    game = matches[num]
    for minute in range(1,game.length()):
        segment_data = game.game_state(minute)
#         print(segment_data)
        if minute == 0 and num == 0:
            data = segment_data
        else:
            data = pd.concat([data,segment_data],ignore_index=True)
print(data)
data.to_csv("small_test_data.csv")

                                                Address  minute  Blue kills  \
0     http://matchhistory.na.leagueoflegends.com/en/...       1           0   
1     http://matchhistory.na.leagueoflegends.com/en/...       2           0   
2     http://matchhistory.na.leagueoflegends.com/en/...       3           0   
3     http://matchhistory.na.leagueoflegends.com/en/...       4           0   
4     http://matchhistory.na.leagueoflegends.com/en/...       5           0   
5     http://matchhistory.na.leagueoflegends.com/en/...       6           0   
6     http://matchhistory.na.leagueoflegends.com/en/...       7           0   
7     http://matchhistory.na.leagueoflegends.com/en/...       8           0   
8     http://matchhistory.na.leagueoflegends.com/en/...       9           0   
9     http://matchhistory.na.leagueoflegends.com/en/...      10           0   
10    http://matchhistory.na.leagueoflegends.com/en/...      11           1   
11    http://matchhistory.na.leagueoflegends.com/en/

In [5]:
# data.to_csv(r'C:\Users\rober\Documents\Data science\datasci-LOL-analysis\data_test.csv', index = False, header=True)

In [6]:
from sklearn.linear_model import LogisticRegression as logreg
# print(titanic[["Pclass_1","Pclass_2","Pclass_3"]])

y = data['Blue Win'].values.reshape(-1, 1)
X = data.drop('Blue Win', axis=1)

# print(X)
# print(y)
mylr = logreg()
mylr.fit(X,y)

C:\Users\rober\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rober\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

class ModelSummary:
    """ This class extracts a summary of the model
    
    Methods
    -------
    get_se()
        computes standard error
    get_ci(SE_est)
        computes confidence intervals
    get_pvals()
        computes p-values
    get_summary(name=None)
        prints the summary of the model
    """
    
    def __init__(self, clf, X, y):
        """
        Parameters
        ----------
        clf: class
            the classifier object model
        X: pandas Dataframe
            matrix of predictors
        y: numpy array
            matrix of variable
        """
        self.clf = clf
        self.X = X
        self.y = y
        pass
    
    def get_se(self):
        """Computes the standard error

        Returns
        -------
            numpy array of standard errors
        """
        # from here https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients
        predProbs = self.clf.predict_proba(self.X)
        X_design = np.hstack([np.ones((self.X.shape[0], 1)), self.X])
        V = np.diagflat(np.product(predProbs, axis=1))
        covLogit = np.linalg.inv(np.dot(np.dot(X_design.T, V), X_design))
        return np.sqrt(np.diag(covLogit))

    def get_ci(self, SE_est):
        """Computes the confidence interval

        Parameters
        ----------
        SE_est: numpy array
            matrix of standard error estimations
        
        Returns
        -------
        cis: numpy array
            matrix of confidence intervals
        """
        p = 0.975
        df = len(self.X) - 2
        crit_t_value = stats.t.ppf(p, df)
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        upper = coefs + (crit_t_value * SE_est)
        lower = coefs - (crit_t_value * SE_est)
        cis = np.zeros((len(coefs), 2))
        cis[:,0] = lower
        cis[:,1] = upper
        return cis
    
    def get_pvals(self):
        """Computes the p-value

        Returns
        -------
        p: numpy array
            matrix of p-values
        """
        # from here https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
        p = self.clf.predict_proba(self.X)
        n = len(p)
        m = len(self.clf.coef_[0]) + 1
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        se = self.get_se()
        t =  coefs/se  
        p = (1 - stats.norm.cdf(abs(t))) * 2
        return p
    
    def get_summary(self, names=None):
        """Prints the summary of the model

        Parameters
        ----------
        names: list
            list of the names of predictors
        """
        ses = self.get_se()
        cis = self.get_ci(ses)
        lower = cis[:, 0]
        upper = cis[:, 1]
        pvals = self.get_pvals()
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        data = []
        for i in range(len(coefs)):
            currlist = []
            currlist.append(np.round(coefs[i], 3))
            currlist.append(np.round(ses[i], 3))
            currlist.append(np.round(pvals[i], 3))
            currlist.append(np.round(lower[i], 3))
            currlist.append(np.round(upper[i], 3))
            data.append(currlist)
        cols = ['coefficient', 'std', 'p-value', '[0.025', '0.975]']
        sumdf = pd.DataFrame(columns=cols, data=data)
        if names is not None:
            new_names = ['intercept']*(len(names) + 1)
            new_names[1:] = [i for i in names]
            sumdf.index = new_names
        else:
            try:
                names = list(self.X.columns)
                new_names = ['intercept']*(len(names) + 1)
                new_names[1:] = [i for i in names]
                sumdf.index = new_names
            except:
                pass
        print(sumdf)
        acc = accuracy_score(self.y, self.clf.predict(self.X))
        confmat = confusion_matrix(self.y, self.clf.predict(self.X))
        print('-'*60)
        print('Confusion Matrix (total:{}) \t Accuracy: \t  {}'.format(len(self.X),np.round(acc, 3)))
        print('  TP: {} | FN: {}'.format(confmat[1][1],confmat[1][0]))
        print('  FP: {} | TN: {}'.format(confmat[0][1],confmat[0][0]))

In [8]:
modsummary = ModelSummary(mylr, X, y)
modsummary.get_summary()

                      coefficient    std  p-value  [0.025  0.975]
intercept                   0.679  0.146    0.000   0.394   0.964
minute                     -0.067  0.017    0.000  -0.100  -0.034
Blue kills                  0.598  0.073    0.000   0.454   0.741
Blue Deaths                -0.731  0.087    0.000  -0.901  -0.561
Kill freq                  -0.047    NaN      NaN     NaN     NaN
Blue kill freq              0.004    NaN      NaN     NaN     NaN
Blue death freq            -0.051    NaN      NaN     NaN     NaN
Red structures taken        0.233  0.084    0.005   0.069   0.397
Blue structures lost        0.226  0.093    0.015   0.043   0.408
Structure freq              0.009    NaN      NaN     NaN     NaN
Blue structure freq         0.075    NaN      NaN     NaN     NaN
Red structure freq         -0.067    NaN      NaN     NaN     NaN
Gold diff                   0.000  0.000    0.000   0.000   0.000
Delta Golld diff           -0.000  0.000    0.420  -0.000   0.000
----------

C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [35]:
import random

# test = matches[12]
# print(random.randint(0,test.length()))
num_test_games = 50
test_data = None
# for num in range(num_games+1,num_test_games+num_games+1):
#     print("test")


for num in range(num_games+1,num_test_games+num_games+1):
    game = matches[num]
    for minute in range(1,game.length()):
        test_segment_data = game.game_state(minute)
#         print(segment_data)
        if minute == 0 and num == num_games+1:
            test_data = test_segment_data
        else:
            test_data = pd.concat([test_data,test_segment_data],ignore_index=True)
# for num in range(num_games+1,num_test_games):
#     game = matches[num]
#     for minute in range(game.length()):
#         test_segment_data = game.game_state(minute)
#         if i == 0 and num == num_games+1:
#             test_data = test_segment_data
#         else:
#             test_data = pd.concat([test_data,test_segment_data],ignore_index=True)
        
print(test_data)
# print(data)

      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
5          6           1            0        0.2             0.2   
6          7           1            0        0.2             0.2   
7          8           1            0        0.2             0.2   
8          9           1            0        0.2             0.2   
9         10           1            0        0.2             0.2   
10        11           1            0        0.0             0.0   
11        12           1            0        0.0             0.0   
12        13           1            0        0.0             0.0   
13        14           1            0        0.0

In [36]:
test_X = test_data.drop('Blue Win', axis=1)
real_y = test_data['Blue Win'].values.reshape(-1, 1)

print(test_X)
print(real_y)


      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
5          6           1            0        0.2             0.2   
6          7           1            0        0.2             0.2   
7          8           1            0        0.2             0.2   
8          9           1            0        0.2             0.2   
9         10           1            0        0.2             0.2   
10        11           1            0        0.0             0.0   
11        12           1            0        0.0             0.0   
12        13           1            0        0.0             0.0   
13        14           1            0        0.0

In [37]:
y_predict = mylr.predict(test_X)
print(y_predict)

[ True  True  True ...  True  True  True]


In [38]:
test_summary = ModelSummary(mylr, test_X, real_y)
test_summary.get_summary()

                      coefficient    std  p-value  [0.025  0.975]
intercept                   0.679  0.147    0.000   0.390   0.968
minute                     -0.067  0.017    0.000  -0.101  -0.033
Blue kills                  0.598  0.063    0.000   0.473   0.722
Blue Deaths                -0.731  0.067    0.000  -0.862  -0.599
Kill freq                  -0.047    NaN      NaN     NaN     NaN
Blue kill freq              0.004    NaN      NaN     NaN     NaN
Blue death freq            -0.051    NaN      NaN     NaN     NaN
Red structures taken        0.233  0.118    0.048   0.002   0.464
Blue structures lost        0.226  0.118    0.055  -0.005   0.456
Structure freq              0.009    NaN      NaN     NaN     NaN
Blue structure freq         0.075    NaN      NaN     NaN     NaN
Red structure freq         -0.067    NaN      NaN     NaN     NaN
Gold diff                   0.000  0.000    0.000   0.000   0.000
Delta Golld diff           -0.000  0.000    0.392  -0.000   0.000
----------

C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\rober\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [39]:
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_X,real_y)))
score = 0
for result in range(len(real_y)):
    if real_y[result] == y_predict[result]:
        score += 1
accuracy = score/len(real_y)
print(accuracy)

0.7454835281615303
